In [27]:
import os
import operator
from typing import Annotated, List, TypedDict
from langchain_openai import ChatOpenAI
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

# set your API key
os.environ["OPENAI_API_KEY"] = "" # مفتاح OpenRouter الخاص بك

# shared state
class TeamState(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]
    next_step: str

# models
llm_smart = ChatOpenAI(model="meta-llama/llama-3.1-70b-instruct", openai_api_base="https://openrouter.ai/api/v1", temperature=0)
llm_fast = ChatOpenAI(model="google/gemini-2.0-flash-001", openai_api_base="https://openrouter.ai/api/v1", temperature=0.7)

# helper to format messages
def format_messages(messages):
    out = []
    for m in messages:
        if isinstance(m, HumanMessage):
            out.append(HumanMessage(content=m.content))
        elif isinstance(m, SystemMessage):
            out.append(SystemMessage(content=m.content))
        else:
            out.append(HumanMessage(content=str(m)))
    return out

# researcher node
def researcher_node(state: TeamState):
    print("🔎 Researcher: gathering facts...")
    msgs = state["messages"]
    resp = llm_smart.invoke(
        [SystemMessage(content="You are a Researcher. Provide detailed facts and key points.")] + format_messages(msgs)
    )
    return {"messages": [resp]}

# writer node
def writer_node(state: TeamState):
    print("✍️ Writer: writing post...")
    msgs = state["messages"]
    resp = llm_fast.invoke(
        [SystemMessage(content="You are a Writer. Write a blog post based on Researcher's findings.")] + format_messages(msgs)
    )
    return {"messages": [resp]}

# supervisor node
def supervisor_node(state: TeamState):
    print("🤖 Supervisor: deciding next step...")
    prompt = """You manage a team: [Researcher, Writer].
Decide next step: 'Researcher', 'Writer', or 'FINISH'."""
    msgs = state["messages"]
    try:
        chain = (
            ChatPromptTemplate.from_messages([("system", prompt), ("placeholder", "{messages}")])
            | llm_smart
            | StrOutputParser()
        )
        next_step = chain.invoke(state).strip()
    except:
        next_step = ""

    # fallback logic
    if next_step not in ["Researcher", "Writer", "FINISH"]:
        last_msg = msgs[-1]
        ai_count = sum(1 for m in msgs if isinstance(m, BaseMessage) and m.type == "ai")
        if ai_count <= 1:
            next_step = "Researcher" if len(str(last_msg.content)) < 200 else "Writer"
        elif ai_count == 2:
            next_step = "FINISH"
        else:
            next_step = "FINISH"

    print(f"👉 Next: {next_step}")
    return {"next_step": next_step}

# decide function
def decide_next_node(state):
    return state["next_step"] if state["next_step"] in ["Researcher", "Writer", "FINISH"] else "FINISH"

# build workflow
workflow = StateGraph(TeamState)
workflow.add_node("Supervisor", supervisor_node)
workflow.add_node("Researcher", researcher_node)
workflow.add_node("Writer", writer_node)
workflow.set_entry_point("Supervisor")
workflow.add_conditional_edges("Supervisor", decide_next_node, {"Researcher":"Researcher","Writer":"Writer","FINISH":END})
workflow.add_edge("Researcher", "Supervisor")
workflow.add_edge("Writer", "Supervisor")

# memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

# run
if __name__ == "__main__":
    cfg = {"configurable": {"thread_id":"demo_run_1"}}
    topic = "The benefits of sleep for programmers"
    print(f"🚀 Starting Task: {topic}")
    msg = HumanMessage(content=f"Research and write a short post about: {topic}")
    for e in app.stream({"messages":[msg]}, config=cfg):
        pass

    print("\n✅ Process Completed!")
    final_state = app.get_state(cfg)
    ai_msgs = [m for m in final_state.values["messages"] if hasattr(m,"content")]
    print("\n--- Final Result ---")
    print(ai_msgs[-1].content if ai_msgs else "⚠️ No AI message found!")


🚀 Starting Task: The benefits of sleep for programmers
🤖 Supervisor: deciding next step...
👉 Next: Researcher
🔎 Researcher: gathering facts...
🤖 Supervisor: deciding next step...
👉 Next: Writer
✍️ Writer: writing post...
🤖 Supervisor: deciding next step...
👉 Next: FINISH

✅ Process Completed!

--- Final Result ---
## Level Up Your Code: Why Sleep is a Programmer's Best Friend

We all know the stereotype: the programmer fueled by caffeine, burning the midnight oil to squash that last bug. But what if I told you the secret to better code, increased productivity, and a sharper mind wasn't another energy drink, but a good night's sleep?

That's right. Sleep isn't just for the weak; it's a crucial tool in a programmer's arsenal. Let's dive into why:

**The Benefits of Shut-Eye for Code Ninjas:**

*   **Sharper Focus, Fewer Bugs:** Think of your brain as a computer. Overclocking it (pulling all-nighters) leads to errors. Sleep allows your brain to process information and consolidate memories